In [0]:
import pandas as pd

In [0]:
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-TensorFlow-Workshop/master/Chapter06/dataset/connect-4.csv'

In [0]:
data = pd.read_csv(file_url)
data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6,c1,c2,c3,c4,c5,c6,d1,d2,d3,d4,d5,d6,e1,e2,e3,e4,e5,e6,f1,f2,f3,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,1,2,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [0]:
target = data.pop('class')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [0]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [0]:
tf.random.set_seed(8)

In [0]:
def model_builder(hp):
  model = tf.keras.Sequential()

  hp_units = hp.Int('units', min_value=128, max_value=512, step=64)

  reg_fc1 = Dense(hp_units, input_shape=(42,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.0001))
  reg_fc2 = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.0001))
  reg_fc3 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.0001))
  reg_fc4 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.0001))
  reg_fc5 = Dense(3, activation='softmax')
  
  model.add(reg_fc1)
  model.add(reg_fc2)
  model.add(reg_fc3)
  model.add(reg_fc4)
  model.add(reg_fc5)
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  hp_learning_rate = hp.Choice('learning_rate', values = [0.01, 0.001, 0.0001]) 

  optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
  model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
  
  return model

In [0]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 5)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [0]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

INFO:tensorflow:Oracle triggered exit


In [0]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters()[0]

In [0]:
best_units = best_hps.get('units')
best_units

192

In [0]:
best_lr = best_hps.get('learning_rate')
best_lr

0.001

In [0]:
model = tuner.hypermodel.build(best_hps)

In [0]:
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Epoch 1/5
1689/1689 [==============================] - 10s 6ms/step - loss: 0.7384 - accuracy: 0.7147 - val_loss: 0.6485 - val_accuracy: 0.7501
Epoch 2/5
1689/1689 [==============================] - 9s 6ms/step - loss: 0.6104 - accuracy: 0.7695 - val_loss: 0.6185 - val_accuracy: 0.7700
Epoch 3/5
1689/1689 [==============================] - 10s 6ms/step - loss: 0.5602 - accuracy: 0.7906 - val_loss: 0.5605 - val_accuracy: 0.7859
Epoch 4/5
1689/1689 [==============================] - 10s 6ms/step - loss: 0.5278 - accuracy: 0.8039 - val_loss: 0.5785 - val_accuracy: 0.7886
Epoch 5/5
1689/1689 [==============================] - 10s 6ms/step - loss: 0.5027 - accuracy: 0.8153 - val_loss: 0.5327 - val_accuracy: 0.8058
